### Required Packages

#### Table of Contents

    1.0 Loading the Dataset
    
    1.1 Working without Duplicates
        1.1.1 Selecting Distinct Observations by removing duplicates.
        1.1.2 Counting distinct records.
        1.1.3 Selecting Top 10.
        1.1.4 Selecting with multiple conditions.
        1.1.5 Selecting only the required columns.
        
    1.2 Using Where Statement as a Filter
        1.2.1 Where statement
        1.2.3 Having as an extra filter
    
    1.3 Tables Joinings and Merging
        1.3.1 Inner Join
        1.3.2 Left Join
        1.3.3 Right Join
        1.3.4 Self Join
        1.3.5 Union
        1.3.6 Union All
    
    1.4 Creating new features using Case Statement 
    
    1.5 Sort Records
        1.5.1 Sorting Records in Ascending order
        1.5.2 Sorting Records in 
        
    1.6 Matching Certain String
        1.6.1 Containing
        1.6.2 Like
        1.6.3 Startwith
        1.6.4 Endwith
        
    1.7 Converting a table from Flat to Wide uisng Case statement.
    
    1.8 Using Group by statement
    
    1.9 Window Functions
    


### 1.0 Loading the Datasets

In [ ]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import json
import os
from sklearn.datasets import fetch_california_housing
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Create a function to glimpse the data
def glimpse(df):
    display(df.info())
    display(df.head())
    display(df.describe())
    #print(f"{df.shape[0]} rows and {df.shape[1]} columns")

def check_missing_data(df):
    missing_columns = df.columns[df.isnull().any()].tolist()
    if len(missing_columns) > 0:
        print("Columns with missing data:")
        for col in missing_columns:
            missing_rows = df[col].isnull().sum()
            print(f"{col}: {missing_rows} missing rows")
    else:
        print("No columns have missing data.")

#### Datasets for Use

In [ ]:
# Load data (will download the data if it's the first time loading)
housing = fetch_california_housing(as_frame=True)
# Create a dataframe
df_0 = housing['data'].join(housing['target'])
glimpse(df_0)
check_missing_data(df_0)

In [ ]:
df_1 = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/nba.csv")
# dropping null value columns to avoid errors
df_1.dropna(inplace = False)
glimpse(df_1)
check_missing_data(df_1)

In [ ]:
url = 'http://universities.hipolabs.com/search?'
# Load the first sheet of the JSON file into a data frame
df_2 = pd.read_json(url, orient='columns').applymap(str)
df_2 = df_2.applymap(str)
glimpse(df_2)
check_missing_data(df_2)

In [ ]:
# select * from df_3 where upper(substring(name,2,2)) = 'ER'  
# substring to be searched
sub ='er'

# start var
start = 2

# creating and passing series to new column
df_2["Indexes"] = df_2["name"].str.find(sub, start)
 
# display
df_2.head()

In [ ]:
# select * from df_2 where upper(substring(name,2,2)) = 'ER'

result = df_2[df_2['name'].str[1:3].str.upper() == 'ER']
result.head()

In [ ]:
#  select * from df_3 where upper(name) like %COLLEGE% and country <> 'United States'

result = df_2[(df_2['name'].str.upper().str.contains('COLLEGE')) & (df_2['country'] != 'United States')]
result.head()

### 1.1 Working without duplicates

#### 1.1.1 selecting distinct observation from the record

In [ ]:
#select distinct name, country, alpha_two_code from df_3
ds_df = df_2[['country','alpha_two_code','name']].drop_duplicates()
ds_df.head()
len(ds_df.name)

In [ ]:
# The fact that distinct name is chosen 
sqldf("select count(name) as vol from (select distinct name, country, alpha_two_code from df_2)")

In [ ]:
sqldf("select count(distinct name) as total from df_2")

In [ ]:
sqldf("select * from df_3")

In [ ]:
Name	Team	Number	Position	Age	Height	Weight	College	Salary

#### 1.1.2 Counting distinct records

In [ ]:
#select country, count(country) as total from df_dist order by total desc
df_3['country'].value_counts()

In [ ]:
sqldf("select country, count(*) as Total from df_3 group by country order by Total desc")

In [ ]:
#select country, count (*) as total from (select distinct name, country, alpha_two_code from df_3) group by country order by total desc
out = df_3[['country','alpha_two_code','name']].drop_duplicates()
out['country'].value_counts()

In [ ]:
sqldf("select country, count(distinct name) as Total from df_3 group by country order by Total desc")

In [ ]:
#To see all of the records of the duplicates
#select country, count(*) as total from (select distinct name, country, alpha_two_code from df_3) group by country having total > 1 
t1 = df_3[df_3['country']=='United States']
t1['name'].value_counts()[t1['name'].value_counts()>1]

In [ ]:
df_3[(df_3['country']=='United States') & ((df_3['name']=='Highland Community College') | (df_3['name']=='Southwestern Community College')) ]

In [ ]:
sqldf("select * from df_3 where country = 'United States' and name in ('Highland Community College','Southwestern Community College')")

#### 1.1.3 Selecting Top 10 countries by number of Universities without duplicates

In [ ]:
#select country, count(country) as counts from df_dist order by counts desc limit 10
t_0 = df_3[['country','alpha_two_code','name']].drop_duplicates()
t_0 = t_0['country'].value_counts().rename_axis('country').reset_index(name='counts')
t_0['percent'] = t_0['counts']/t_0['counts'].sum()
t_0['percent'] = t_0['percent'].map('{:.2%}'.format)
t_0.head(10)

In [ ]:
sqldf("select country,count")

#### 1.1.4 selecting a variable and calculating the mean

Using Python

In [ ]:
#select Team, avg(Salary) as AvgSalary from df_2 where Salary != '' group by Team order by AvgSalary desc Limit 10;
t_f = df_2[['Team','Salary']].groupby('Team').mean().dropna().reset_index()
t_f.sort_values(by=['Salary'], inplace=True, ascending=False)
t_f.head(10)

In [ ]:
sqldf("select team,count(*)as vol,sum(salary) as total_sal, avg(Salary) as avg_sal from df_2 where Salary != '' group by Team order by avg_sal desc limit 10")

In [ ]:
#select Team, Name, max(Salary) as Salary from df_2 group by Team,Name order by Salary desc limit 10
tt = df_2[['Name','Team','Salary']].groupby(['Team','Name']).max().dropna().reset_index()
tt.sort_values(by=['Salary'], inplace=True, ascending=False)
tt.head(10)

In [ ]:
sqldf("select Team, Name, max(Salary) as Salary from df_2 group by Team,Name order by Salary desc limit 10")

In [ ]:
#select max(Salary) from df_2 where age = 20
df_2[df_2.Team == 'New York Knicks']['Salary'].max()

In [ ]:
sqldf("select max(salary) as Max_Sal from df_2")

In [ ]:
#which team has the highest 

#### 1.3.1 Inner Join using a Key

In [ ]:
df_inner = pd.merge(left=df, right=df_1, left_on='ClientID', right_on='ClientID')

In [ ]:
df_inner.head()

In [ ]:
list(df_inner)

#### *** Selecting only the required columns

In [ ]:
df_inner_1 = df_inner[['ClientID','LoanDate','LoanAmount','Term','LoanSeries','LoanPeriodID','RepaymentTerm','InterestAmount',
 'Age','year_with_the_bank','default','predicted_scores','Defaulted_0','Defaulted_1']]

In [ ]:
df_inner_1.head()

In [ ]:
df_inner_1.to_csv('C:\\Users\\seune\\desktop\\base_scoring_1.csv',index=False)

#### *** Selecting with multiple conditions.

In [ ]:
#df_sql = np.where((df['LoanDate']>='8/8/2017') & (df['LoanAmount'] > 50000) & (df['FT_Team'].str.startswith('S')))

#Select * from df where LoanDate >= '8/8/2017' and LoanAmount > 50000 and Age > 30
df_sql = np.where((df['LoanDate'] >= '8/8/2017') & (df['LoanAmount'] > 50000) & (df['Age'] > 50))


In [ ]:
df_sql_1 = df.loc[df_sql]
df_sql_1.head()

In [ ]:
df_sql_1.shape

In [ ]:
# sorting dataframe
df_3.sort_values("Team", inplace = True)

df_3.dropna(inplace = True)
  
# making boolean series for a team name
filter1 = df_3["Team"]=="Atlanta Hawks"
  
# making boolean series for age
filter2 = df_3["Age"]>=27
  
# filtering data on basis of both filters
df_3.where(filter1 & filter2, inplace = True)

#.dropna(inplace = True)
  
# display
df_3

In [ ]:
#select * from df_3 where Team = 'Atlanta Hawks'

# sorting dataframe
df_3.sort_values("Team", inplace = True)
  
# making boolean series for a team name
filter = df_3["Team"]=="Atlanta Hawks"
  
# filtering data
df_3.where(filter, inplace = True)
  
# display
df_3

#### *** Selecting only records with Wildcard 

In [ ]:
df_distinct.head()

#### *** Where a strings starts with U

In [ ]:
#select all record from the dataframe where name like University
df_json_sql = np.where((df_distinct['alpha_two_code'] == 'US') & (df_distinct['name'].str.startswith('U')))

In [ ]:
df_json_sql_1 = df_distinct.loc[df_json_sql]
df_json_sql_1.head()

In [ ]:
df_json_sql_1.shape

In [ ]:
#select all record from the dataframe where name ends with 'Uni'
df_json1 = np.where((df_distinct['alpha_two_code'] == 'US') & (df_distinct['name'].str.endswith('Uni')))
df_json_1 = df_distinct.loc[df_json1]

#### *** Where a string contains 'Uni' and 'Was'

In [ ]:
#select all record from the dataframe where name like Uni and name like Was
df_json_s = np.where((df_json['name'].str.contains('Uni')) & (df_json['name'].str.contains('Was')))

In [ ]:
df_json_s_1 = df_json.loc[df_json_s]
df_json_s_1.head()

In [ ]:
df_json_s_a = df_json[df_json['name'].str.contains('Uni') & df_json['name'].str.contains('Was')]
df_json_s_a.head()

#### *** Select Country, alpha_two_code and count distinct Universities in each of the countries 

In [ ]:
#df_json.loc['country','alpha_two_code']

df_3[['country','alpha_two_code']].agg(['nunique','count','size'])


#.agg(['nunique','count','size'])

In [ ]:
df_3.groupby(['country','alpha_two_code']).agg(['count', 'nunique']).stack()

### ****Case when 

In [ ]:
def myfunc(Start_Date,End_Date):
    Full_Year_Goal = None
    if (Start_Date >= '2014-07-27' or Start_Date == '2014-07-27') and End_Date <= '2015-07-25':
        Full_Year_Goal = 'Y'
    elif (Start_Date == '2015-07-26' or Start_Date == '2015-07-27') and End_Date == '2016-07-30':
        Full_Year_Goal = 'Y'
    elif (Start_Date == '2016-07-31' and End_Date == '2017-07-29'):
        Full_Year_Goal = 'Y'
    elif (Start_Date == '2017-07-30' or Start_Date == '2017-07-31') and End_Date == '2018-07-28':
        Full_Year_Goal = 'Y'
    else:
        Full_Year_Goal = 'N'
    return Full_Year_Goal

In [ ]:
myfunc(Start_Date = '2014-07-27',End_Date = '2015-07-22')